In [22]:
import os
import tqdm
import os.path as osp
import pandas as pd
%run env.py
%run utils/integration.py
article_data_file = osp.join(DATA_DIR, 'articles', 'data.csv')
output_corpus_dir = osp.join(DATA_DIR, 'articles', 'corpus', 'corpus_00', 'docs')

In [13]:
df = pd.read_csv(article_data_file)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3500 entries, 0 to 3499
Data columns (total 6 columns):
abstract    3481 non-null object
date        2155 non-null object
has_text    3500 non-null bool
id          3500 non-null int64
title       3500 non-null object
xml         3500 non-null object
dtypes: bool(1), int64(1), object(4)
memory usage: 140.2+ KB


In [17]:
text = []
for i, r in tqdm.tqdm(df.iterrows(), len=len(df)):
    body = extract_text(r['xml'])
    text.append(combine_text(r['title'], r['abstract'], body))
df['text'] = text

100%|██████████| 3500/3500 [02:46<00:00, 21.05it/s]


In [24]:
if not osp.exists(output_corpus_dir):
    os.makedirs(output_corpus_dir)

LIMIT = 500
df_exp = df.sample(n=LIMIT, random_state=1)

for i, r in tqdm.tqdm(df_exp.iterrows()):
    path = osp.join(output_corpus_dir, 'PMC{}.txt'.format(r['id']))
    with open(path, 'w') as fd:
        fd.write(r['text'])
output_corpus_dir

500it [00:00, 1631.91it/s]


'/Users/eczech/data/research/hammer/nlp/20190311-pubmed-tcell-relation/articles/corpus/corpus_00'